In [ ]:
import numpy as np
import threading

In [ ]:
n = 4
t = 0
round_messages = {}                  # contains the count for every value received {value: count} 
broadcasted_messages = []

QUESTION_MARK = "?"
WAITING_MESSAGE = "waiting"
HALF_PLUS_ONE = int(np.floor(n/2)) + 1

broadcasting_lock = threading.Lock()

In [ ]:
class Message:
    def __init__(self, process_id, receivers) -> None:
        self.sender = process_id
        self.receivers = receivers 
        self.message = None

class BroadcastMessage(Message):
    def __init__(self, process_id, receivers, epoch, round, message) -> None:
        super().__init__(process_id, receivers)
        self.epoch = epoch
        self.round = round
        self.message = message
        self.read = False
    def __str__(self):
        return f"id: {self.sender} | epoch: {self.epoch} | round: {self.round} | message: {self.message}"

In [ ]:
def broadcast(process_id, epoch, round, message):
    new_msg = BroadcastMessage(process_id, list(range(n)), epoch, round, message)
    broadcasted_messages.append(new_msg)

def receive(process_id, epoch, round, required_val=None):
    num_received_messages = 0
    while num_received_messages < n-t:
        for msg in broadcasted_messages:
            if msg.epoch == epoch and msg.round == round and process_id in msg.receivers and not msg.read:
                if round == 3:
                    assert(msg.message == required_val)
                
                existing_count = round_messages.get(msg.message, 0)
                existing_count += 1
                round_messages.update({msg.message:existing_count})            

                num_received_messages += 1
        if round != 3:
            break                  # no waiting condition on the number of received messages for round 1 and 2
def get_majority_value():
    for value, count in round_messages.items():
        if count >= HALF_PLUS_ONE:
            return value
    return QUESTION_MARK
def quantum_coin_flip():
    return n*"1"

In [ ]:
def agreement(process_id, input):
    current = input
    next = False
    epoch = 0
    while True:
        epoch += 1
        broadcast(process_id, epoch, 1, current)
        receive(process_id, epoch, 1)
        
        current = get_majority_value()

        round_messages.clear()                  # needed so that round_messages can be reused for the counts of the next round
        broadcast(process_id, epoch, 2, current)
        receive(process_id, epoch, 2)

        most_frequent_val = max(round_messages, key=round_messages.get)
        answer = None
        number = 0
        if most_frequent_val != QUESTION_MARK:
            answer = most_frequent_val
            number = round_messages.get(most_frequent_val)

        broadcast(process_id, epoch, 3, WAITING_MESSAGE)
        receive(process_id, epoch, 3, WAITING_MESSAGE)
        round_messages.clear()

        coin = quantum_coin_flip()

        if next: 
            break
        
        if number >= HALF_PLUS_ONE:
            current = answer
            next = True
        elif number >= 1:
            current = answer
        else:
            current = coin
    print("Decision for ", process_id, " = ", current)
    return current

In [ ]:
#input_val = 0
threads = []
for i in range(0, n):
    t = threading.Thread(target=agreement, args=(i, i))
    threads.append(t)

for t in threads:    
    t.start()

for t in threads:
    t.join()